In [8]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [9]:
#fname = '../experiment3_1/simulated1_c4_e0.01_k21.dist'
fname = '../data/histograms/e_c4_k21.hist'

In [10]:
import sys
sys.path.append('..')
from covest import *
from models import *
hist_orig, hist = load_hist(fname)

In [11]:
hist

[0,
 12059395,
 828270,
 247429,
 61049,
 13661,
 3892,
 1891,
 1225,
 942,
 676,
 393,
 236,
 111,
 91,
 53,
 29,
 10,
 13,
 4,
 3,
 0,
 2,
 0,
 2,
 2,
 3,
 3,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 1,
 2,
 1,
 5,
 1,
 4,
 1,
 4,
 3,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 0]

In [12]:
model = RepeatsModel(21, 100, hist)

In [13]:
model.compute_loglikelihood(10, 0.05, 0.9, 0.9, 0.9)

-5398132.095137997

In [16]:
import multiprocessing

try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default
    
pool = multiprocessing.Pool(processes=cpus)

In [15]:
from functools import partial
count = 10
base_c = 5
range_c = 5
step_c = range_c/count
base_e = 0.1
range_e = 0.1
step_e = range_e/count

c_vals = arange(base_c - range_c, base_c + range_c, step_c)
e_vals = arange(base_e - range_e, base_e + range_e, step_e)

q1, q2, q = 0.9, 0.9, 0.9

def compute_loglikelihood(c, e):
    return model.compute_loglikelihood(c, e, q1, q2, q)

# f1 = np.array([[compute_loglikelihood(c, e) for e in e_vals] for c in c_vals])
f1 = array([list(
        pool.map(partial(compute_loglikelihood, c), arange(base_e - range_e, base_e + range_e, step_e))
    ) for c in arange(base_c - range_c, base_c + range_c, step_c)
])
f1[isinf(f1)] = NaN


Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiproce

KeyboardInterrupt: 

In [ ]:
def transform_for_visualisation(a):
    min_val, max_val = nanmin(f1), nanmax(f1)    
    print(min_val - max_val - 1)
    def transform(x):
        return log(1 - ((x - min_val) / (max_val - min_val + 1)))
    vtransform = vectorize(transform)
    return vtransform(a)

In [ ]:
ax = imshow(
    transform_for_visualisation(f1), interpolation='none', cmap = 'jet',
    extent=[e_vals[0],e_vals[-1],c_vals[-1],c_vals[0]], aspect='auto',
)
colorbar(ax)
show()